In [ ]:
%load_ext autoreload
%autoreload 2
import cv2
from scipy.ndimage import distance_transform_edt as dist
import IPython.display 
import importlib
import skimage.io as imgio
import numpy as np
import os
import torch
import torch.nn.functional as F
import math
import re
from matplotlib import pyplot as plt
from scipy import ndimage
from topoloss4neurons.networks import UNet
import networktraining as nt
# from topoloss4neurons import load_dataset
%matplotlib inline
def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)
    
def showCube(vol):
    v1=np.max(vol,axis=0)
    v2=np.max(vol,axis=1)
    v3=np.max(vol,axis=2)
    imshow(v1*255)
    imshow(v2*255)
    imshow(v3*255) 

In [ ]:
def volInds(swcCoords,scale,volDims,offset,downsampling):
    """
    For scaling and shifting coordinates
    """
    x=int((swcCoords[volDims[0]]*scale[0]+offset[0])*downsampling[0])
    y=int((swcCoords[volDims[1]]*scale[1]+offset[1])*downsampling[1])
    z=int((swcCoords[volDims[2]]*scale[2]+offset[2])*downsampling[2])
    return x,y,z

In [ ]:
def traceLine(lbl,begPoint,endPoint):
    """
    Renders a line represented by two nodes
    """
    d=endPoint-begPoint
    s=begPoint
    mi=np.argmax(np.fabs(d))
    coef=d/d[mi]
    sz=np.array(lbl.shape)
    numsteps=int(abs(d[mi]))+1
    step=int(d[mi]/abs(d[mi]))
    for t in range(0,numsteps):
        pos=np.array(s+coef*t*step)
        if np.all(pos<sz) and np.all(pos>=0):
            #print(pos)
            lbl[tuple(pos.astype(np.int))]=1
#         else:
#             print("reqested point",pos,"but the volume size is",sz)
    return lbl


In [ ]:
def readSWC(brain_i):
    """
    Returns the full annotation information for the given brain index
    """
    if brain_i == 6:
        swcfname = "/cvlabdata2/home/oner/CarlsData/AL066-AL_stamp_2019_07_23_10_34.ano.eswc"
    elif brain_i == 8:
        swcfname = "/cvlabdata2/home/oner/CarlsData/AL080_stamp_2020_01_22_13_26.ano.eswc"
    elif brain_i == 9:
        swcfname = "/cvlabdata2/home/oner/CarlsData/AL092_stamp_2020_01_09_11_10.ano.eswc_sorted.eswc"
    elif brain_i == 175:
        swcfname = "/cvlabdata2/home/oner/CarlsData/annotations/AL175_stamp_2021_02_10_13_52.ano.eswc"
    elif brain_i == 223:
        swcfname = None
        return {0:[6833,7549,488],1:[3091,7665,410],2:[7088,7804,554],3:[3346,7920,665]}
    elif brain_i == 230:
        swcfname = None
        return {0:[3935,8628,288],1:[4109,8883,543],2:[3713,8397,389],3:[3968,8653,644]}
    else:
        return
    nodes=dict()
    for a in open(swcfname):
        if (re.match('\s*\#',a)!=None):
#             print("commment line", a)
            continue
        b=a.split()
        c=map(lambda x: float(x), b)
        d=list(c)
        nodes[int(d[0])]=d
    return nodes

def getCoords(brain_i):
    """
    Returns the location of nodes in the annotations
    """
    nodes = readSWC(brain_i)
    coords = []
    for k in nodes :
        n=nodes[k]
        if brain_i == 6:
            x,y,z= volInds(n,[1,1,1],[2,3,4], [0,0,0], [1,1,1])
        elif brain_i == 8:
            x,y,z= volInds(n,[1,1,1],[2,3,4], [0,0,0], [1,1,1])
        elif brain_i == 9:
            x,y,z= volInds(n,[1,1,1],[2,3,4], [0,0,0], [1,1,1])
        elif brain_i == 175:
            x,y,z= volInds(n,[1,1,1],[2,3,4], [0,0,0], [1,1,1])
        elif brain_i == 223:
            x,y,z= volInds(n,[1,1,1],[0,1,2], [0,0,0], [1,1,1])
        elif brain_i == 230:
            x,y,z= volInds(n,[1,1,1],[0,1,2], [0,0,0], [1,1,1])
        coords.append([x,y,z])
    return coords

def renderSWC2volume(swcfname, volumeDims, volCL, scale, offset, downsampling):
    '''
      swcfname      name of the swc file
      volumeDims    one-dimensional array;
                    volumeDims[1] is index of volCL dimension corresponding to X
                    volumeDims[2] is index of volCL dimension corresp to Y
                    volumeDims[3] is index of volCL dimension corresp to Z
                    X,Y,Z are as interpreted in the CWS format
      volCL         np array into which we will render ground truth centerlines
    '''
    distthresh=40
    nodes=dict()
    for a in open(swcfname):
        if (re.match('\s*\#',a)!=None):
#             print("commment line", a)
            continue
        b=a.split()
        c=map(lambda x: float(x), b)
        d=list(c)
        nodes[int(d[0])]=d
        #print(d)
    # start here
    for k in nodes :
        n=nodes[k]
        x,y,z= volInds(n,scale,volumeDims, offset, downsampling)
        parent=nodes.get(int(n[6]),None)
        #print(n)
        if parent!=None :
            #print(n,parent)
            xp,yp,zp=volInds(parent,scale,volumeDims, offset, downsampling)
            #print(x,y,z,volCL.shape)
            #print(xp,yp,zp,volCL.shape)
            if (x!=xp or y!=yp or z!=zp) and ((abs(x-xp)+abs(y-yp)+abs(z-zp))<distthresh):
                #print("line: ({},{},{})-({},{},{})".format(xp,yp,zp,x,y,z))
                traceLine(volCL,np.array([xp,yp,zp]),np.array([x,y,z]))
        

In [ ]:
def getCubeCoords(brain_i):
    """
    Returns xyz borders of the cubes according to terafiles
    """
    if brain_i == 6:
        direc = "/cvlabdata2/home/oner/CarlsData/6RES(11711x16382x2000)/"
    elif brain_i == 8:
        direc = "/cvlabdata2/home/oner/CarlsData/8RES(11692x19566x1600)/"
    elif brain_i == 9:
        direc = "/cvlabdata2/home/oner/CarlsData/9RES(11692x16123x1700)/"
    elif brain_i == 175:
        direc = "/cvlabdata2/home/oner/CarlsData/AL175/RES(15186x17117x1919)"
    elif brain_i == 223:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL223/"
    elif brain_i == 230:
        direc = "/home/oner/samba/TeraConvertion/AL230/RES(14504x17408x2468)/"
    else:
        return
    yind = sorted([int(x)//10 for x in os.listdir(direc) if x.startswith("0") or x.startswith("1")] )
    xind = sorted([int(x.split("_")[-1])//10 for x in os.listdir(os.path.join(direc,"000000")) if x.startswith("0") or x.startswith("1")] )
    zind = sorted([int(x.split("_")[-1][:-4])//10 for x in os.listdir(os.path.join(direc,"000000", "000000_000000")) if x.startswith("0") or x.startswith("1")] )
    
    return xind, yind, zind

In [ ]:
import os
import tifffile
def getCube(brain_i, cube):
    """
    Returns the image of the desired cube in the desired brain
    """
    if brain_i == 6:
        direc = "/home/oner/samba/Ter/6RES(11711x16382x2000)/"
    elif brain_i == 8:
        direc = "/cvlabdata2/home/oner/CarlsData/8RES(11692x19566x1600)/"
    elif brain_i == 9:
        direc = "/cvlabdata2/home/oner/CarlsData/9RES(11692x16123x1700)/"
    elif brain_i == 175:
        direc = "/cvlabdata2/home/oner/CarlsData/AL175/RES(15186x17117x1919)"
    elif brain_i == 223:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL223/"
    elif brain_i == 230:
        direc = "/home/oner/samba/TeraConvertion/AL230/RES(14504x17408x2468)/"
    else:
        return
    xs,ys,zs = getCubeCoords(brain_i)
    x,y,z = cube
    l1 = "{:06d}".format(ys[y]*10)
    l2 = "{:06d}_{:06d}".format(ys[y]*10,xs[x]*10)
    l3 = "{:06d}_{:06d}_{:06d}.tif".format(ys[y]*10,xs[x]*10,zs[z]*10)
    volume = tifffile.imread(os.path.join(direc,l1,l2,l3))
    return volume.transpose((1,2,0))

def getCubename(brain_i, cube):
    """
    Returns the directory for the desired cube
    """
    if brain_i == 6:
        direc = "/cvlabdata2/home/oner/CarlsData/6RES(11711x16382x2000)/"
    elif brain_i == 8:
        direc = "/cvlabdata2/home/oner/CarlsData/8RES(11692x19566x1600)/"
    elif brain_i == 9:
        direc = "/cvlabdata2/home/oner/CarlsData/9RES(11692x16123x1700)/"
    elif brain_i == 175:
        direc = "/cvlabdata2/home/oner/CarlsData/AL175/RES(15186x17117x1919)"
    elif brain_i == 223:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL223/"
    elif brain_i == 230:
        direc = "/home/oner/samba/TeraConvertion/AL230/RES(14504x17408x2468)/"
    else:
        return
    xs,ys,zs = getCubeCoords(brain_i)
    x,y,z = cube
    l1 = "{:06d}".format(ys[y]*10)
    l2 = "{:06d}_{:06d}".format(ys[y]*10,xs[x]*10)
    l3 = "{:06d}_{:06d}_{:06d}.tif".format(ys[y]*10,xs[x]*10,zs[z]*10)
    
    return os.path.join("/datasets/" + direc.split("/")[-2],l1,l2,l3)

def getLabel(brain_i,cube):
    """
    Returns the label of the desired cube in the desired brain
    """
    xs, ys, zs = getCubeCoords(brain_i)
    xi, xf = xs[cube[0]:cube[0]+2]
    yi, yf = ys[cube[1]:cube[1]+2]
    zi, zf = zs[cube[2]:cube[2]+2]
    m=[1,1,1]
    o=[-int(yi),-int(xi),-int(zi)]
    scale_factor=[1,1,1]

    downsampling=torch.tensor(scale_factor,dtype=torch.double)
    offset=torch.tensor(o,dtype=torch.double)
    scale=torch.tensor(m,dtype=torch.double)
    volDims=torch.tensor([3,2,4],dtype=torch.long)
    vols = np.zeros((yf-yi,xf-xi,zf-zi), dtype=np.uint8)
    if brain_i == 6:
        renderSWC2volume(swcname1, volDims, vols, scale, offset, downsampling)
    elif brain_i == 8:
        renderSWC2volume(swcname2, volDims, vols, scale, offset, downsampling)
    elif brain_i == 9:
        renderSWC2volume(swcname3, volDims, vols, scale, offset, downsampling)
    elif brain_i == 175:
        renderSWC2volume(swcname4, volDims, vols, scale, offset, downsampling)
    else:
        return
    return vols

In [ ]:
def findCubes(brain_i):
    """
    If the annotation is provided, returns the indices of cubes which contains annotated pixels
    """
    xs, ys, zs = getCubeCoords(brain_i)
    coords = getCoords(brain_i)
    cubes = []
    if len(coords) == 0:
        cubes = np.array(np.meshgrid(np.arange(len(xs)),np.arange(len(ys)),np.arange(len(zs)))).T.reshape((-1,3))
    for c in coords:
        xi = np.where(c[0] > np.array(xs))[0][-1]
        yi = np.where(c[1] > np.array(ys))[0][-1]
        zi = np.where(c[2] > np.array(zs))[0][-1]
        if [xi,yi,zi] not in cubes:
            cubes.append([xi,yi,zi])
    return cubes

In [ ]:
def loadPred(brain_i, cube):
    """
    Returns the prediction of the desired cube in the desired brain
    """
    if brain_i == 6:
        direc = "/cvlabdata2/home/oner/CarlsData/6RES(11711x16382x2000)/"
    elif brain_i == 8:
        direc = "/cvlabdata2/home/oner/CarlsData/8RES(11692x19566x1600)/"
    elif brain_i == 9:
        direc = "/cvlabdata2/home/oner/CarlsData/9RES(11692x16123x1700)/"
    elif brain_i == 175:
        direc = "/cvlabdata2/home/oner/CarlsData/AL175/RES(15186x17117x1919)/"
    elif brain_i == 223:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL223/"
    elif brain_i == 230:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL230/"
    else:
        return
    xs,ys,zs = getCubeCoords(brain_i)
    x,y,z = cube
    l1 = "{:06d}".format(ys[y]*10)
    l2 = "{:06d}_{:06d}".format(ys[y]*10,xs[x]*10)
    l3 = "{:06d}_{:06d}_{:06d}.npy".format(ys[y]*10,xs[x]*10,zs[z]*10)
    pred = np.load(os.path.join(direc,l1,l2,l3)).transpose(2,0,1)
    return pred

def loadPredf(brain_i, cube):
    """
    Returns the prediction_f of the desired cub in the desired brain
    """
    if brain_i == 6:
        direc = "/cvlabdata2/home/oner/CarlsData/6RES(11711x16382x2000)/"
    elif brain_i == 8:
        direc = "/cvlabdata2/home/oner/CarlsData/8RES(11692x19566x1600)/"
    elif brain_i == 9:
        direc = "/cvlabdata2/home/oner/CarlsData/9RES(11692x16123x1700)/"
    elif brain_i == 175:
        direc = "/cvlabdata2/home/oner/CarlsData/AL175/RES(15186x17117x1919)/"
    elif brain_i == 223:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL223/"
    elif brain_i == 230:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL230/"
    else:
        return
    xs,ys,zs = getCubeCoords(brain_i)
    x,y,z = cube
    l1 = "{:06d}".format(ys[y]*10)
    l2 = "{:06d}_{:06d}".format(ys[y]*10,xs[x]*10)
    l3 = "{:06d}_{:06d}_{:06d}_f.npy".format(ys[y]*10,xs[x]*10,zs[z]*10)
    pred = np.load(os.path.join(direc,l1,l2,l3)).transpose(2,0,1)
    return pred

def savePred(brain_i, cube, pred):
    """
    Saves the prediction for the desired cub in the desired brain
    """
    if brain_i == 6:
        direc = "/cvlabdata2/home/oner/CarlsData/6RES(11711x16382x2000)/"
    elif brain_i == 8:
        direc = "/cvlabdata2/home/oner/CarlsData/8RES(11692x19566x1600)/"
    elif brain_i == 9:
        direc = "/cvlabdata2/home/oner/CarlsData/9RES(11692x16123x1700)/"
    elif brain_i == 175:
        direc = "/cvlabdata2/home/oner/CarlsData/AL175/RES(15186x17117x1919)/"
    elif brain_i == 223:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL223/"
    elif brain_i == 230:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL230/"
    else:
        return
    xs,ys,zs = getCubeCoords(brain_i)
    x,y,z = cube
    l1 = "{:06d}".format(ys[y]*10)
    l2 = "{:06d}_{:06d}".format(ys[y]*10,xs[x]*10)
    l3 = "{:06d}_{:06d}_{:06d}.npy".format(ys[y]*10,xs[x]*10,zs[z]*10)
    nt.utils.mkdir(os.path.join(direc,l1,l2))
    np.save(os.path.join(direc,l1,l2,l3), pred.transpose(1,2,0))
    
def savePredf(brain_i, cube, pred):
    """
    Saves the prediction_f for the desired cub in the desired brain
    """
    if brain_i == 6:
        direc = "/cvlabdata2/home/oner/CarlsData/6RES(11711x16382x2000)/"
    elif brain_i == 8:
        direc = "/cvlabdata2/home/oner/CarlsData/8RES(11692x19566x1600)/"
    elif brain_i == 9:
        direc = "/cvlabdata2/home/oner/CarlsData/9RES(11692x16123x1700)/"
    elif brain_i == 175:
        direc = "/cvlabdata2/home/oner/CarlsData/AL175/RES(15186x17117x1919)/"
    elif brain_i == 223:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL223/"
    elif brain_i == 230:
        direc = "/cvlabdata2/home/oner/CarlsData/Data/AL230/"
    else:
        return
    xs,ys,zs = getCubeCoords(brain_i)
    x,y,z = cube
    l1 = "{:06d}".format(ys[y]*10)
    l2 = "{:06d}_{:06d}".format(ys[y]*10,xs[x]*10)
    l3 = "{:06d}_{:06d}_{:06d}_f.npy".format(ys[y]*10,xs[x]*10,zs[z]*10)
    nt.utils.mkdir(os.path.join(direc,l1,l2))
    np.save(os.path.join(direc,l1,l2,l3), pred.transpose(1,2,0))

In [ ]:
# swcname="/cvlabdata2/home/kozinski/MBFneuron/DeFelipe_Auto_DK_Edited_Complete.SWC"
swcname1 = "/cvlabdata2/home/oner/CarlsData/AL066-AL_stamp_2019_07_23_10_34.ano.eswc"
swcname2 = "/cvlabdata2/home/oner/CarlsData/AL080_stamp_2020_01_22_13_26.ano.eswc"
swcname3 = "/cvlabdata2/home/oner/CarlsData/AL092_stamp_2020_01_09_11_10.ano.eswc_sorted.eswc"
swcname4 = "/cvlabdata2/home/oner/CarlsData/AL175_stamp_2021_02_10_13_52.ano.eswc"

In [ ]:
nodes = readSWC(175)
coords = getCoords(175)

In [ ]:
xs,ys,zs = getCubeCoords(175)

In [ ]:
cubes = findCubes(175)

In [ ]:
def getNeighbours(cube_i, which_n=[], is_all=False):
    """
    Returns the indices of neighbour cubes of the desired cube
    which_n : If you want to specify which neighbours you want to get
    """
    neighs = []
    if is_all:
        for i in range(6):
            n = [0,0,0]
            if i < 3:
                n[i] = 1
            else:
                n[i%3] = -1
            neighs.append(list(np.array(cube_i) + np.array(n)))
    else:
        for n in which_n:
            neighs.append(list(np.array(cube_i) + np.array(n)))
        return neighs
    
    return neighs

def checkBorders(pred, th=3, border=5):
    """
    For the given prediction check which borders of the cube contains foreground voxels
    th : Threshold to binarize the prediction
    border : width of border region you want to check for foreground voxels
    """
    borders = []
    pred_bin = pred < th
    if np.sum(pred_bin[:border,:,:]) > 0:
        borders.append([0,-1,0])
    if np.sum(pred_bin[:,:border,:]) > 0:
        borders.append([-1,0,0])
    if np.sum(pred_bin[:,:,:border]) > 0:
        borders.append([0,0,-1])
    if np.sum(pred_bin[-border:,:,:]) > 0:
        borders.append([0,1,0])
    if np.sum(pred_bin[:,-border:,:]) > 0:
        borders.append([1,0,0])
    if np.sum(pred_bin[:,:,-border:]) > 0:
        borders.append([0,0,1])
    return borders

def process_chunk(chunk):
    pred = network(chunk)
    return pred

def segmentCube(im, network):
    """
    Segments the given image with the given network
    """
    with nt.torch_no_grad:
        image_i = np.float32(im[None,None])
        image_i  = nt.to_torch(image_i, volatile=True).contiguous().to("cuda")
        out_shape = (image_i.shape[0],1,*image_i.shape[2:])
        pred_i = nt.to_torch(np.empty(out_shape, np.float32), volatile=True).cuda()
        pred_i = nt.process_in_chuncks(image_i, pred_i, process_chunk,
                                       [96,96,96], [12,12,12])
        pred_np = pred_i.detach().cpu().numpy()[0,0]
        return pred_np

In [ ]:
network = UNet(in_channels=1,
                   m_channels=32,
                   out_channels=1,
                   n_convs=2,
                   n_levels=5,
                   dropout=0.1,
                   batch_norm=True,
                   upsampling="deconv",
                   pooling="max",
                   three_dimensional=True).to("cuda")

network.load_state_dict(torch.load("/cvlabdata2/home/oner/CarlsData/new_logs/log_base_5lvl_2convs/network_bestqual.pickle"))
network.eval()

In [ ]:
import time
ttt = time.time()
save = True
# Keeps track of the already segmented cubes so that we don't segment them again
processed_cubes = []
# Cubes in queue for segmenting
pending_cubes = []

## Select random starting cubes
starting_cubes = np.random.choice(len(cubes),4,replace=False)
starting_cubes = [cubes[i] for i in starting_cubes]
# Add all given statring cubes to the queue
for sc in starting_cubes:
    pending_cubes.append(sc)
ii = 0
# Get the coordinates of the cube (jsut for printing)
xs,ys,zs = getCubeCoords(175)

while len(pending_cubes) > 0: # checks if there are any cubes in queue
    current_cube = pending_cubes.pop(0) # get the first cube in queue
    if current_cube in processed_cubes: # check if you already processed this cube
        print(current_cube, " already processed")
        continue
    print("Processing ", current_cube)
    print(xs[current_cube[0]],ys[current_cube[1]],zs[current_cube[2]])
    
    im = getCube(175, current_cube)/65535 # get the image of the cube and normalize the uint16 images
    pred = segmentCube(im, network) # get the prediction

    if save:
        savePred(175, current_cube, pred) # save them if you want
    which_n = checkBorders(pred, th=8, border=8) # check for border voxels and find the neighbours which probably contains neuron
    
    print("Neighbours ", getNeighbours(current_cube, which_n))
    processed_cubes.append(current_cube)
    pending_cubes += getNeighbours(current_cube, which_n) # add the neighbour according to border voxels
    
    if ii % 20:
        plt.figure(figsize=(20,10))
        plt.subplot(121)
        plt.imshow(im.max(2))
        plt.subplot(122)
        plt.imshow(pred.min(2))
        plt.show()
        
print(time.time()-ttt)


## For Visualizing the big predictions

In [ ]:
import skimage.measure

# im2 = skimage.measure.block_reduce(im, (4,4,4), np.max)
# pred2 = skimage.measure.block_reduce(pred, (4,4,4), np.min, cval=15)

In [ ]:
### Select the cubes and display them

In [ ]:
neuron_cubes = []
for i in range(np.min(cubes,axis=0)[0], np.max(cubes,axis=0)[0]+1):
    for j in range(np.min(cubes,axis=0)[1], np.max(cubes,axis=0)[1]+1):
        for k in range(np.min(cubes,axis=0)[2], np.max(cubes,axis=0)[2]+1):
            neuron_cubes.append([i,j,k])

In [ ]:
print(np.array(cubes).max(0) - np.array(cubes).min(0) +1)

In [ ]:
dims = np.array(cubes).max(0) - np.array(cubes).min(0) +1
maxs = np.array(cubes).max(0)
mins = np.array(cubes).min(0)

In [ ]:
## Downscale the cubes by 4 and concat them in an array to visualize it
## We are loading all the cubes in a big rectangle prism which contains the whole neuron
ims = np.zeros((dims[2]*75,dims[0]*75,dims[1]*75))
preds = 15*np.ones((dims[2]*75,dims[0]*75,dims[1]*75))
imss = []
predss = []
x = 0
y = 0
z = 0
ind = 0
for i in range(mins[0],maxs[0]+1):
    for j in range(mins[1],maxs[1]+1):
        for k in range(mins[2],maxs[2]+1):
            print(i,j,k)
            im = np.float32(getCube(175, [i,j,k])/65535)
            im2 = skimage.measure.block_reduce(im, (4,4,4), np.max)
            try: # if it is segmented load the prediction
                pred_np = loadPred(175, [i,j,k])
                pred_np = skimage.measure.block_reduce(pred_np, (4,4,4), np.min, cval=15)
                print("Loaded")
            except: # if it is an empty cube just put all 15 (because the highest value in distance map is 15)
                pred_np = 15*np.ones(im2.shape)

            imss.append(im2)
            predss.append(pred_np)
            shp = im2.shape
            ims[z:z+shp[2],x:x+shp[1],y:y+shp[0]] = im2.transpose(2,1,0)
            preds[z:z+shp[2],x:x+shp[1],y:y+shp[0]] = pred_np.transpose(2,1,0)
            z += shp[2]
        y += shp[0]
        z = 0
    x += shp[1]
    y = 0

In [ ]:
%matplotlib inline

In [ ]:
# ims2 = np.load("im175.npy")
plt.figure(figsize=(20,20))
plt.imshow(ims.max(0))

In [ ]:
# preds2 = np.load("pred175.npy")
plt.figure(figsize=(20,20))
plt.imshow(preds2.min(0))

In [ ]:
np.save("./im175.npy", ims)
np.save("./pred175.npy", preds)